<a href="https://colab.research.google.com/github/sliscak/notebooks/blob/main/Whisper%2BStable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install requirements

In [9]:
!pip install --upgrade diffusers
!pip install --upgrade gradio
!pip install --upgrade ftfy
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


KeyboardInterrupt: ignored

In [ ]:
import gradio as gr
import whisper
import os
from torch import autocast
from diffusers import StableDiffusionPipeline
from google.colab import output
from huggingface_hub import notebook_login

In [ ]:
# login to verify license
notebook_login()

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=True)
pipe = pipe.to("cuda")
model = whisper.load_model("small")

def transcribe(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    language = max(probs, key=probs.get)
    options = whisper.DecodingOptions(language=language)
    prompt = whisper.decode(model, mel, options).text
    with autocast("cuda"):
        image = pipe(prompt).images[0]
    return prompt, image

block = gr.Blocks(css=".container { margin: auto; }")
with block as demo:
    gr.Interface(
        fn=transcribe,
        inputs=gr.Audio(source="microphone", type="filepath"),
        outputs=["text", "image"])

demo.launch(debug=True)